In [1]:
"""
ISC with sliding window to capture variations in relationship between ISC and emotional report
"""

import os
import pickle
from concurrent.futures import ThreadPoolExecutor, as_completed
import cProfile
import time
from glob import glob
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from isc_standalone import p_from_null, isc, load_boolean_mask
from ISC_Helper import get_rois, _compute_phaseshift_sliding_isc, load_roi_data, _compute_sliding_isc, parcellate_bold, load_schaeffer1000, parcel_to_nifti
import nibabel as nib
from nilearn import plotting
from nilearn.masking import unmask
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from copy import deepcopy

# -------------------------------
# Parameters
# -------------------------------
task = 'onesmallstep'
roi_selected = ['wholebrain']
# roi_selected = ['PCC', 'ACC']
emotions = ['P', 'N', 'M', 'X', 'Cry']  # Positive, Negative, Mixed, Neutral
avg_over_roi = True
spatial = False
pairwise = False
random_state = None
window_size = 30
step_size = 5
if task == 'toystory':
    n_trs = 274
    n_shifts = 10000
elif task == 'onesmallstep':
    n_trs = 454
    n_shifts = 10000
else:
    raise Exception('task not defined')
n_windows = int((n_trs - window_size) / step_size) + 1
batch_size = 16

smooth = 'smooth'
avg_over_roi_name = "avg" if avg_over_roi else "voxelwise"
spatial_name = "spatial" if spatial else "temporal"
pairwise_name = "pairwise" if pairwise else "group"

# -------------------------------
# File paths
# -------------------------------
home_dir = "/Volumes/BCI/Ambivalent_Affect/RishabISC/ISC"
label_dir = f"{home_dir}/data/{task}/"

if task == 'toystory':
    data_dir_func = '/Volumes/BCI/Ambivalent_Affect/fMRI_Study/ISC_Data/ToyStoryNuisanceRegressed'
elif task == 'onesmallstep':
    data_dir_func = '/Volumes/BCI/Ambivalent_Affect/fMRI_Study/ISC_Data_cut/NuisanceRegressed'
else:
    raise ValueError('Invalid task')
    
func_fns = glob(join(data_dir_func, 'P?.nii.gz')) + glob(join(data_dir_func, 'N?.nii.gz')) + \
           glob(join(data_dir_func, 'VR?.nii.gz')) + glob(join(data_dir_func, 'P??.nii.gz')) + \
           glob(join(data_dir_func, 'N??.nii.gz')) + glob(join(data_dir_func, 'VR??.nii.gz'))

# if task == 'toystory':
#     # remove VR7 and 8 temporarily for testing because they are 295 not 300 TRs
#     func_fns = [fn for fn in func_fns if 'VR7' not in fn and 'VR8' not in fn]
#     label_dir += "Toy_Story_Labelled"
# elif task == 'onesmallstep':
#     label_dir += "OSS_Labelled"

subj_ids = [str(subj).split('/')[-1].split('.')[0] for subj in func_fns]  # assumes BIDS format
subj_ids.sort()

roi_mask_path = '/Volumes/BCI/Ambivalent_Affect/rois'
all_roi_fpaths = glob(os.path.join(roi_mask_path, '*.nii*'))
all_roi_masker = get_rois(all_roi_fpaths)
data_path = f'{home_dir}/data/{task}'
figure_path = f'{home_dir}/figures/{task}'
parc_path = f"{data_path}/../rois/schaefer_2018/Schaefer2018_1000Parcels_17Networks_order_FSLMNI152_2mm.nii.gz"
mask_path = f"{data_path}/mask_img.npy"
isc_path = f"{data_path}/isc_sliding_{pairwise_name}_n{len(subj_ids)}_{avg_over_roi_name}_roi{len(roi_selected)}_" \
           f"window{window_size}_step{step_size}.pkl"
sliding_perm_path = f"{data_path}/sliding_isc/permutations/phaseshift_size{window_size}_step{step_size}"
save_path = f"{sliding_perm_path}_{n_shifts}perms_{len(roi_selected)}rois"
print(save_path)

/opt/anaconda3/envs/isc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Volumes/BCI/Ambivalent_Affect/RishabISC/ISC/data/onesmallstep/sliding_isc/permutations/phaseshift_size30_step5_10000perms_1rois


## Get Schaeffer 1000 Parcellation

In [2]:
parc_path, mask_path

('/Volumes/BCI/Ambivalent_Affect/RishabISC/ISC/data/onesmallstep/../rois/schaefer_2018/Schaefer2018_1000Parcels_17Networks_order_FSLMNI152_2mm.nii.gz',
 '/Volumes/BCI/Ambivalent_Affect/RishabISC/ISC/data/onesmallstep/mask_img.npy')

In [3]:
masked_parc = load_schaeffer1000(parc_path, mask_path)

In [4]:
masked_parc.shape

(228483,)

In [5]:
# isc_data_path = f"{data_path}/func_data_parcellated"

# if not os.path.exists(isc_data_path):
#     from nilearn.datasets import fetch_atlas_schaefer_2018
#     from nilearn.maskers import NiftiLabelsMasker

#     all_parcels = []
#     n_parcels = 1000
#     atlas = fetch_atlas_schaefer_2018(n_rois=n_parcels, yeo_networks=17, resolution_mm=2, data_dir=data_path)
#     labels = [x.decode('UTF-8') for x in atlas.labels]  # https://stackoverflow.com/questions/23618218/numpy-bytes-to-plain-string

#     # Initialize labels masker with atlas parcels
#     masker = NiftiLabelsMasker(atlas.maps, labels=labels)

#     for file in func_fns[:2]:
#         # Fit masker to extract mean time series for parcels
#         all_parcels.append(masker.fit_transform(file))

#     all_parcels = np.array(all_parcels).transpose(1, 2, 0)
#     np.save(isc_data_path, all_parcels)
# else:
#     # load in parcellated data from file
#     all_parcels = np.load(isc_data_path)


## Load and preprocess BOLD data

In [ ]:
from itertools import repeat
with ThreadPoolExecutor() as executor:
    bold_roi = executor.map(load_roi_data, roi_selected, repeat(all_roi_masker), repeat(func_fns), repeat(data_path))  # repeat is used to pass the parameter to each iteration in map(). the 

KeyboardInterrupt: 

ROI wholebrain, subj #0: P2 loaded from file


In [ ]:
# # wholebrain_paths = glob(join(data_path, "bold_roi", "wholebrain_*"))

# all_parcel_data = []
# for p in wholebrain_paths:
#     z = np.load(p)
#     n_parcels = 1000
#     parcel_ts = np.zeros((z.shape[0], n_parcels))
#     for parcel_id in range(1, n_parcels + 1):
#         parcel_voxels = np.where(masked_parc == parcel_id)[0]
#         if parcel_voxels.size > 0:
#             parcel_ts[:, parcel_id - 1] = np.mean(z[:, parcel_voxels], axis=1)

#     all_parcel_data.append(parcel_ts)


In [ ]:
x = list(bold_roi)

In [ ]:
x[0].shape

(454, 228483, 27)

In [ ]:
roi = parcellate_bold(x[0], 1000, masked_parc)
print(roi.shape)

(454, 228483, 27)


IndexError: index 230786 is out of bounds for axis 1 with size 228483

In [11]:
masked_parc.shape

(369422,)

In [ ]:
# data = np.nan_to_num(data, nan=0.0)  # replace nan with 0
# data = np.clip(data, -1e6, 1e6)  # clip max and min values
# from scipy.stats import zscore
# # data = zscore(data, axis=0)

# all_parcel_data = []
# n_parcels = 1000
# # Initialize output (timepoints, parcels, subjects)
# parcel_ts = np.zeros((data.shape[0], n_parcels, data.shape[2]))

# # Loop over each subject independently
# for subj_idx in range(data.shape[2]):
#     for parcel_id in range(1, n_parcels + 1):
#         parcel_voxels = np.where(masked_parc == parcel_id)[0]
#         if parcel_voxels.size > 0:
#             parcel_ts[:, parcel_id - 1, subj_idx] = np.mean(data[:, parcel_voxels, subj_idx], axis=1)

# all_parcel_data.append(parcel_ts)  # Shape: (454, 1000, 27)

# all_parcel_data = np.array(all_parcel_data)[0]  # Shape: (num_subjects, 454, 1000, 27)


In [ ]:
# all_parcel_data.shape

In [ ]:
isc_data_path = f"{data_path}/bold_roi/all_func_data_parcellated"
# all_parcel_data = np.array(all_parcel_data).transpose(1,2,0)
# if not os.path.exists(isc_data_path):
# np.save(isc_data_path, all_parcel_data)

In [ ]:
# print("NaN count in data:", np.isnan(all_parcel_data).sum())
# print("Inf count in data:", np.isinf(all_parcel_data).sum())
# print("Max value in data:", np.nanmax(all_parcel_data))
# print("Min value in data:", np.nanmin(all_parcel_data))
# all_parcel_data.shape

In [ ]:
# all_parcel_data = zscore(all_parcel_data, axis=0)
# all_parcel_data.shape

In [ ]:
# print("NaN count in data:", np.isnan(all_parcel_data).sum())
# print("Inf count in data:", np.isinf(all_parcel_data).sum())
# print("Max value in data:", np.nanmax(all_parcel_data))
# print("Min value in data:", np.nanmin(all_parcel_data))
# all_parcel_data.shape

## Compute ISC and Visualize

In [ ]:
# compute standard temporal ISC parcelwise using a leave-one-out approach
parcel_isc = isc(all_parcel_data)
# np.save(f"{isc_data_path}_isc", parcel_isc)

In [ ]:
parcel_isc.shape

In [ ]:
mask_name = f"{roi_mask_path}/wholebrain.nii.gz"

# code from https://brainiak.org/tutorials/10-isc/
# Load the brain mask
brain_mask = load_boolean_mask(mask_name)

# Get the list of nonzero voxel coordinates
coords = np.where(brain_mask)

brain_nii = nib.load(mask_name)

In [ ]:
x = parcel_isc.mean(axis=0)
isc_volume = np.zeros((91, 109, 91))

# Fill each parcel with its ISC value
for parcel_id in range(1, 1001):
    isc_volume[parc.get_fdata() == parcel_id] = x[parcel_id - 1]

isc_nii = nib.Nifti1Image(isc_volume, parc.affine)

In [ ]:
from nilearn import plotting
f, ax = plt.subplots(1,1, figsize = (12, 5))
plotting.plot_stat_map(
    isc_nii, 
    axes=ax,
    threshold=0.1
)
plt.show()

In [ ]:
isc_nii.shape

In [ ]:
# nib.save(isc_nii, f"{data_path}/mean_isc_1000parcels")

In [ ]:
label_dir

In [ ]:
# load behavioral data
coded_states = np.load(f'{label_dir}/coded_states_{task}.npy')
print('shape before trimming:', coded_states.shape)
if task == 'onesmallstep':
    coded_states = coded_states[:, :-30]
elif task == 'toystory':
    coded_states = coded_states[:, :-26]    
    
print('shape after trimming:', coded_states.shape)


In [ ]:
timepoint_variance = np.var(coded_states[:, :n_trs, :], axis=0)  # shape=(n_trs, n_emotions)

# Initialize sliding window output
slide_behav = np.zeros((n_windows, timepoint_variance.shape[1]))

# Calculate mean variance within each sliding window
for i in range(n_windows):
    start_idx = i * step_size
    end_idx = start_idx + window_size
    slide_behav[i] = np.mean(timepoint_variance[start_idx:end_idx], axis=0)

In [ ]:
print('shape before removing:', slide_behav.shape)  # 8 rois, shape=(n_windows, n_emotions)
# remove crying and neutral to just focus on pos, neg, mix, for 8 rois
slide_behav = slide_behav[:, :3]
# slide_behav = np.delete(slide_behav, 3, axis=1)  # remove neutral
emotions = ['P', 'N', 'M', 'X', 'Cry']
emotions = emotions[:3]
print('shape after removing:', slide_behav.shape)

In [ ]:
slide_behav.shape

In [ ]:
all_parcel_data.transpose(2,0,1).shape

In [ ]:
isc_parcel_mean = isc(all_parcel_data.transpose(2,0,1), summary_statistic='median')
isc_parcel_mean.shape

In [ ]:
all_parcel_data.shape

In [ ]:
tolerate_nans=True
summary_statistic='median'
avg_over_roi=False
observed = _compute_sliding_isc(all_parcel_data, n_trs, window_size, step_size, avg_over_roi, spatial=spatial,
                                    pairwise=pairwise, summary_statistic=summary_statistic, tolerate_nans=tolerate_nans)


In [ ]:
observed.shape

In [ ]:
brain_nii.shape

In [ ]:
isc_brain = np.zeros((*brain_nii.shape, n_windows))
isc_brain.shape

In [ ]:
data_path

In [ ]:
for p in range(1, n_parcels + 1):
    mask = parc.get_fdata() == p  # location of current parcel
    isc_brain[mask, :] = observed[:, p - 1].T

isc_nifti = nib.Nifti1Image(isc_brain, brain_nii.affine)
# nib.save(isc_nifti, f"{data_path}/isc_window")

In [ ]:
slide_behav[:,2, np.newaxis].shape

In [ ]:
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(slide_behav[:,2,np.newaxis])
# X_scaled = scaler_X.fit_transform(slide_behav)

scaler_Y = StandardScaler()
Y_scaled = scaler_Y.fit_transform(observed)

ridge = RidgeCV(alphas=np.logspace(-6, 6, 13), store_cv_values=True)

# Fit ridge regression per parcel
betas = np.zeros((1000, 1))
# betas = np.zeros((1000, 3))
for parcel in range(Y_scaled.shape[1]):
    ridge.fit(X_scaled, Y_scaled[:, parcel])
    betas[parcel, :] = ridge.coef_

In [ ]:
emotion_betas = np.zeros((*parc.shape, 1))
# emotion_betas = np.zeros((*parc.shape, 3))

for p in range(1, n_parcels + 1):  # Parcels are labeled from 1 to 1000
    mask = parc.get_fdata() == p  # location of current parcel
    emotion_betas[mask, :] = betas[p - 1, :]

beta_nifti = nib.Nifti1Image(emotion_betas, brain_nii.affine)
# nib.save(beta_nifti, f"{data_path}/emotion_betas_mix_only")